In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

import sys
sys.path.append('../')
from utils import *

In [2]:
df = pd.read_pickle('../../data/datasets/video-level-watches+autoencoded-upnext')
videos = pd.read_pickle('../../data/videos_raw_metadata')
title_embeddings = pd.read_pickle('../../data/embeddings/title-')

2023-07-15 22:03:51.629559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-15 22:03:51.629606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-15 22:03:54.103824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-15 22:03:54.103878: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-15 22:03:54.103927: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (toutatis): /proc/driver/nvidia/version does not exist
2023-07-15 22:03:54.104516: I tensorflow/core/platform/cpu_feature_g

In [3]:
X = []
Y = []
for watch in tqdm(df):
    playing = watch['playing']
    selected = watch['selected']
    if playing not in videos or selected not in videos or any([s not in videos for s in watch['upnext']]):
        continue

    if len(watch['upnext']) < 15:
        continue
        
    upnext = watch['upnext'][:15]
    for s in upnext:
        playing = watch['playing']
        selected = watch['selected']

        if s == selected:
            y = 1
        else:
            y = 0

        playing = np.array(title_embeddings[videos[playing]['snippet']['title']])[0]
        suggested = np.array(title_embeddings[videos[s]['snippet']['title']])[0]
        all_suggested = [np.array(title_embeddings[videos[s]['snippet']['title']])[0] for s in upnext]
        
        # all_suggested = np.array(watch['upnext_embs'])[0]

        x = np.array([playing, suggested, all_suggested]).T

        X.append(x)
        Y.append(y)

  0%|          | 0/6050 [00:00<?, ?it/s]

In [4]:
X = np.array(X)
Y = np.array(Y)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

In [6]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [7]:
pos = Y.sum()
neg = len(Y) - pos
total = neg + pos
w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)
class_weights = {0:w0, 1:w1}
class_weights

{0: 0.534742570113018, 1: 7.69578313253012}

In [8]:
X_train.shape

(55188, 128, 3)

In [10]:
# CNN model to predict if a video is selected from the upnext list
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras import optimizers
from keras import regularizers
import keras

# CNN model

# Input
input_shape = (128, 3)
input_layer = Input(shape=input_shape)

# Convolutional layers
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
conv1 = MaxPooling1D(pool_size=2)(conv1)
conv1 = Dropout(0.2)(conv1)

conv2 = Conv1D(filters=64, kernel_size=3, activation='relu')(conv1)
conv2 = MaxPooling1D(pool_size=2)(conv2)
conv2 = Dropout(0.2)(conv2)

conv3 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv2)
conv3 = MaxPooling1D(pool_size=2)(conv3)
conv3 = Dropout(0.2)(conv3)

# Flatten
flat = Flatten()(conv3)

# Dense layers
hidden1 = Dense(128, activation='relu')(flat)
hidden1 = Dropout(0.2)(hidden1)

hidden2 = Dense(64, activation='relu')(hidden1)
hidden2 = Dropout(0.2)(hidden2)

# Output
output = Dense(1, activation='sigmoid')(hidden2)

METRICS = [keras.metrics.BinaryAccuracy(name='accuracy'), keras.metrics.AUC(name='auc')]

model = Model(inputs=input_layer, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

model.fit(X_train, y_train, epochs=10, batch_size=64, class_weight=class_weights)

Epoch 1/10
863/863 [==============================] - 28s 31ms/step - loss: 0.6948 - accuracy: 0.4725 - auc: 0.5016
Epoch 2/10
863/863 [==============================] - 25s 29ms/step - loss: 0.6935 - accuracy: 0.4516 - auc: 0.4929
Epoch 3/10
863/863 [==============================] - 25s 29ms/step - loss: 0.6934 - accuracy: 0.4117 - auc: 0.4938
Epoch 4/10
863/863 [==============================] - 24s 28ms/step - loss: 0.6934 - accuracy: 0.4941 - auc: 0.4966
Epoch 5/10
863/863 [==============================] - 24s 28ms/step - loss: 0.6934 - accuracy: 0.3992 - auc: 0.4949
Epoch 6/10
863/863 [==============================] - 24s 27ms/step - loss: 0.6935 - accuracy: 0.6826 - auc: 0.4905
Epoch 7/10
863/863 [==============================] - 24s 28ms/step - loss: 0.6933 - accuracy: 0.6735 - auc: 0.4946
Epoch 8/10
863/863 [==============================] - 24s 27ms/step - loss: 0.6933 - accuracy: 0.2900 - auc: 0.4932
Epoch 9/10
863/863 [==============================] - 24s 27ms/step - lo

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from matplotlib import pyplot

yhat = model.predict_proba(testX)
pos_probs = yhat[:, 1]
pyplot.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
fpr, tpr, _ = roc_curve(testy, pos_probs)
pyplot.plot(fpr, tpr, marker='.', label='Logistic')
# axis labels
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()